# Projeto 1 - Ciência dos Dados

Nome: Enzo Qental

Nome: João Pedro Artese de Natal

Nome: João Pedro Ramos Fiorin

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [76]:
# Importando as funções que serão utilizadas no nosso classificador.
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import nltk 
import numpy
nltk.download('rslp')

[nltk_data] Downloading package rslp to C:\Users\Eduardo
[nltk_data]     Natal\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [47]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Eduardo Natal\Desktop\c dados\cdados p1\CDADOS-P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [48]:
filename = 'Iphone1250.xlsx'

In [49]:
# Leitura do excel da planilha treinamento. 
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,"eu achava iphone caro,até pagar o preço de nun...",1
1,"novamente sem carregador, essa vida usuária de...",2
2,se fôssemos trocar de celular vocês ficariam c...,2
3,"@zaynikr mas gente, ela tem uns 3 telefones co...",2
4,esse face id do iphone que se eu tiver com a b...,3


In [50]:
# Leitura do excel da planilha teste.
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,"voltei c iphone, que sdd que eu estava meu deus",1
1,o novo ios do iphone ta muito bom,3
2,"so pq tem um iphone, é triste viu kkkkkjjjjj h...",0
3,carregador de iphone seria perfeito se fosse m...,3
4,ela vive dando reply nos story usando iphone k...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Celular da Apple. Consideramos como uma postagem relevante quando o conteúdo se trata de alguma crítica ou análise sobre o produto, caso o contrário, consideramos como não relevante.

___
### Montando as funções de limpeza dos twits.

Criando as funções que irão limpar os nossos dados das duas planilhas(Treinamento e Teste).

In [51]:
# Lista de todas stopwords, que serão retiradas no cleanup.
stopwords=['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vós', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'esteja', 'estejamos', 'sejam', 'fosse', 'fôssemos', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 'seriam', 'tenho', 'tem', 'temos', 'têm', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam']

In [52]:
# Criação da função cleanup.
def cleanup(text):
    punctuation = '[!-.:?;()\'",_%$\\|/,<>[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text).lower()
    x=[p for p in text_subbed.split() if '@' != p [0] and p not in stopwords]
    stemmer = nltk.stem.RSLPStemmer()
    x = list(map(stemmer.stem, x))
    return x

In [53]:
# Aplicando a função cleanup na planilha de treinamento.
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train

,Treinamento,Relevância
0,"[ach, iphon, car, pag, preç, nunc, am, verdad]",1
1,"[nov, carreg, vid, usuár, iphon, pra, mim, 😭😭😭]",2
2,"[troc, celul, fic, iphon, xr]",2
3,"[gent, um, 3, telefon, diss, cert, iphon, kkkk...",2
4,"[fac, id, iphon, boc, tiquin, abert, reconhec,...",3
...,...,...
995,"[iphon, xr, 89, bat, amig, vend, inform, cham,...",0
996,"[namor, deu, iphon, part, trá, vidr, agor, cui...",1
997,"[saudad, to, iphon, surre, 😫]",1
998,"[quer, cri, vsc, log, iphon, tens, to, armazen]",2


In [54]:
# Aplicando a função cleanup na planilha de teste.
test['Teste'] = test['Teste'].apply(cleanup)
test

,Teste,Relevância
0,"[volt, c, iphon, sdd, deu]",1
1,"[nov, io, iphon, ta, bom]",3
2,"[so, pq, iphon, trist, viu, kkkkkjjjjj, http, ...",0
3,"[carreg, iphon, perfeit, barat]",3
4,"[viv, dand, reply, story, us, iphon, kkkkkkkk,...",2
...,...,...
245,"[instal, whatsapp, gb, watus, iphon, jailbreak...",0
246,"[vid, mud, vinic, desativ, espelh, automá, iphon]",1
247,"[bat, celul, tá, parec, iphon, tão, merd, q, tá]",3
248,"[narcis, dá, iphon, 🥺]",0


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [55]:
# Transformnado train['Relevância'] em variável categórica.
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = [0,1,2,3]

In [56]:
# Separando os tweets por relevância.
muito_relevante = train['Relevância'] == 3
relevante = train['Relevância'] == 2
irrelevante = train['Relevância'] == 1
muito_irrelevante = train['Relevância'] == 0

In [57]:
# Separando os tweets por relevância em variáveis.
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

In [58]:
# Colocando todas as palavras, por categoria, em uma lista.
total_palavras_mr = sum(df_muito_relevante['Treinamento'], [])
total_palavras_r = sum(df_relevante['Treinamento'], [])
total_palavras_i = sum(df_irrelevante['Treinamento'], [])
total_palavras_mi = sum(df_muito_irrelevante['Treinamento'], [])

In [59]:
# Transformando as palavras da lista em um Pandas Series. 
series_mr = pd.Series(total_palavras_mr)
series_r = pd.Series(total_palavras_r)
series_i = pd.Series(total_palavras_i)
series_mi = pd.Series(total_palavras_mi)

In [60]:
# Calculando o total de palavras, sem repetições, de todas as categorias.
tot_palavras_sem_rep = len(series_mr.value_counts()) + len(series_r.value_counts()) + len(series_i.value_counts()) + len(series_mi.value_counts())

In [61]:
# Total de palavras por categoria. 
total_mr = len(total_palavras_mr)
total_r = len(total_palavras_r)
total_i = len(total_palavras_i)
total_mi = len(total_palavras_mi)

In [62]:
# Probabilidade de cada categoria: 
prob_mr = len(df_muito_relevante)/len(train["Treinamento"])
prob_r = len(df_relevante)/len(train["Treinamento"])
prob_i = len(df_irrelevante)/len(train["Treinamento"])
prob_mi = len(df_muito_irrelevante)/len(train["Treinamento"])

In [63]:
# Criando o classificador.
def classifica(frase):
    # Suavização de la Place.
    alfa = 1 
    # Listas por categorias para armazenar a probabilidade de cada palavra da frase para serem multiplicadas posteriormente.
    lista_mr = []
    lista_r = []
    lista_i = []
    lista_mi = []
    # Loop principal.
    for palavra in frase:
        # Quantidade de aparições da palavra em cada categoria.
        qtd_mr = series_mr.to_list().count(palavra)
        qtd_r = series_r.to_list().count(palavra)
        qtd_i = series_i.to_list().count(palavra)
        qtd_mi = series_mi.to_list().count(palavra)
        # Calculos da probabilidade por categoria utilizando a suavização de La Place.
        prob_palavra_mr = lista_mr.append((qtd_mr + alfa) / (total_mr + tot_palavras_sem_rep))
        prob_palavra_r = lista_r.append((qtd_r + alfa) / (total_r + tot_palavras_sem_rep))
        prob_palavra_i = lista_i.append((qtd_i + alfa) / (total_i + tot_palavras_sem_rep))
        prob_palavra_mi = lista_mi.append((qtd_mi + alfa) / (total_mi + tot_palavras_sem_rep))
    # Multiplicação das probabilidades por palavra, pela probabilidade da categoria.        
    prob_total_mr = numpy.prod(lista_mr) * prob_mr
    prob_total_r = numpy.prod(lista_r) * prob_r
    prob_total_i = numpy.prod(lista_i) * prob_i
    prob_total_mi = numpy.prod(lista_mi) * prob_mi
    # Lista que armazena todas as probabilidades totais por categoria.
    lista_prob_total = [prob_total_mr,prob_total_r,prob_total_i,prob_total_mi]
    # Análise da maior probabilidade, e retorno da função.
    if max(lista_prob_total) == prob_total_mr:
        return 3
    elif max(lista_prob_total) == prob_total_r:
        return 2
    elif max(lista_prob_total) == prob_total_i:
        return 1
    elif max(lista_prob_total) == prob_total_mi:
        return 0

In [64]:
# Aplicação da função na planilha de treinamento.
train['Classificacao'] = train['Treinamento'].apply(classifica)

In [65]:
# Fazendo o crosstab para comparar a relevância e classificação da planilha treinamento.
tabela = pd.crosstab(train['Relevância'], train['Classificacao'], normalize='all')
tabela

Classificacao,0,1,2,3
Relevância,,,,
0,0.426,0.003,0.001,0.00
1,0.051,0.229,0.006,0.00
2,0.039,0.016,0.153,0.00
3,0.019,0.017,0.020,0.02


In [66]:
# Eficácia do classificador na planilha de treinamento.
f'{(tabela.iloc[0,0] + tabela.iloc[1,1] + tabela.iloc[2,2] + tabela.iloc[3,3])*100:.2f}%'

'82.80%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [67]:
# Transformando test['Relevância'] em categórica.
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = [0,1,2,3]

In [68]:
# # Aplicação da função na planilha de teste.
test['Classificacao'] = test['Teste'].apply(classifica)

In [69]:

# # Fazendo o crosstab para comparar a relevância e classificação da planilha de teste.
tabela_test = pd.crosstab(test['Relevância'], test['Classificacao'], normalize='all')
tabela_test

Classificacao,0,1,2
Relevância,,,
0,0.252,0.084,0.016
1,0.156,0.076,0.020
2,0.120,0.052,0.044
3,0.092,0.036,0.052


In [70]:
# Eficácia do classificador na planilha de teste.
f'Nosso classificador acertou:{(tabela_test.iloc[0,0] + tabela_test.iloc[1,1] + tabela_test.iloc[2,2] + 0)*100:.2f}%'

'Nosso classificador acertou:37.20%'

___
### Verificando performance do classificador

In [71]:
# Porcentagem de verdadeiros positivos.
print(f'Porcentagem de mensagens muito relevantes e que foram classificadas como muito relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens relevantes e que foram classificadas como relevantes:{(tabela_test.iloc[2,2])*100:.2f}%')

Porcentagem de mensagens muito relevantes e que foram classificadas como muito relevantes:0.00%

Porcentagem de mensagens relevantes e que foram classificadas como relevantes:4.40%


In [72]:
# Porcentagem de falsos positivos.
print(f'Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens muito irrelevantes e que foram classificadas como relevantes:{(tabela_test.iloc[0,2])*100:.2f}%')
print('')
print(f'Porcentagem de mensagens irrelevantes e que foram classificadas como muito relevantes:{0*100:.2f}%')
print('')
print(f'Porcentagem de mensagens irrelevantes e que foram classificadas como relevantes:{(tabela_test.iloc[1,2])*100:.2f}%')

Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito relevantes:0.00%

Porcentagem de mensagens muito irrelevantes e que foram classificadas como relevantes:1.60%

Porcentagem de mensagens irrelevantes e que foram classificadas como muito relevantes:0.00%

Porcentagem de mensagens irrelevantes e que foram classificadas como relevantes:2.00%


In [73]:
# Porcentagem de verdadeiros negativos.
print(f'Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito irrelevantes:{(tabela_test.iloc[0,0])*100:.2f}%')
print('')
print(f'Porcentagem de mensagens irrelevantes e que foram classificadas como irrelevantes:{(tabela_test.iloc[1,1])*100:.2f}%')

Porcentagem de mensagens muito irrelevantes e que foram classificadas como muito irrelevantes:25.20%

Porcentagem de mensagens irrelevantes e que foram classificadas como irrelevantes:7.60%


In [74]:
# Porcentagem de falsos negativos.
(f'Porcentagem de mensagens muito relevantes e que foram classificadas como muito irrelevantes:{(tabela_test.iloc[3,0])*100:.2f}%')
print('')
print(f'Porcentagem de mensagens muito relevantes e que foram classificadas como irrelevantes:{(tabela_test.iloc[3,1])*100:.2f}%')
print('')
print(f'Porcentagem de mensagens relevantes e que foram classificadas como muito irrelevantes:{(tabela_test.iloc[2,0])*100:.2f}%')
print('')
print(f'Porcentagem de mensagens relevantes e que foram classificadas como irrelevantes:{(tabela_test.iloc[2,1])*100:.2f}%')

Porcentagem de mensagens muito relevantes e que foram classificadas como muito irrelevantes:9.20%

Porcentagem de mensagens muito relevantes e que foram classificadas como irrelevantes:3.60%

Porcentagem de mensagens relevantes e que foram classificadas como muito irrelevantes:12.00%

Porcentagem de mensagens relevantes e que foram classificadas como irrelevantes:5.20%


In [75]:
# Eficácia do classificador na planilha de testes.
print('Acurácia total: ')
print(f'{(tabela_test.iloc[0,0] + tabela_test.iloc[1,1] + tabela_test.iloc[2,2] + 0)*100:.2f}%')


Acurácia total: 
37.20%


___
### Concluindo


#### Explicação de como são tratadas as mensagens com dupla negação e sarcasmo.
O classificador Naïve-Bayes funciona através do reconhecimento de palavras e cálculo de suas probabilidades. Por conta disso afirma-se que não é possivel analisar se um frase possui sarcasmo ou dupla negação. 


#### Porque devem continuar financiando nosso projeto?
O nosso projeto deve ser financiado, tendo em vista que com o decorrer do tempo, a nossa base de dados aumenta, influenciando tambem na melhoria de nossa precisão e exatidão.



#### Porque não se pode alimentar a própria base de treinamento usando o classificador aplicado a novos tweets?
Nós não podemos utilizar o próprio classificador para alimentar a base de treinamentos, porque a função que classifica foi programada a partir das classificações de treinamentos. Sendo assim, a cada classificação nova, o modelo da função é alterado, fazendo com que nunca se resulte em algo padronizado.


#### Diferentes cenários de uso para o classificador Naïve-Bayes.
O classificador probabilístico Naïve-Bayes caracteriza-se por ser muito útil para se realizar análises automáticas de qualquer produto,pessoa,marca... Portanto é possivel utilizá-lo em diversos cenários diferentes,por exemplo as diversas classificações feitas pelo IBGE, como renda per capita, estimativa de vida, estimativa de população até uma certa data etc...

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**